In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score

/usr/lib64/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


164.77874541282654


In [4]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.001943826675415039


In [5]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.5698015689849854


In [6]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

3.9291863441467285


In [7]:
#remove columns which have any null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

50.37196207046509


In [8]:
len(unclean_columns)

36

In [9]:
unclean_columns

Index(['installation_date', 'service_contract_last_renewal_date',
       'service_contract_classification', 'contract_line_duration_in_days',
       'marketing_program_name', 'service_contract_discount_percentage',
       'pos_bookings_measure_transaction_type', 'pos_adjustment_type',
       'cisco_one_opportunity_type', 'ios_software_version',
       'service_offering', 'service_program', 'service_fee_amount',
       'last_date_of_support', 'end_new_service_attachment_date',
       'end_of_service_contract_renewal_date', 'end_software_maintenance_date',
       'service_level_group', 'mapped_service_list_price',
       'mapped_service_level', 'service_group', 'service_brand_code',
       'service_product_base_service_fee_amount',
       'service_product_base_last_date_of_support',
       'service_product_base_end_new_service_attachment_date',
       'service_product_base_end_of_service_contract_renewal_date',
       'service_product_base_end_software_maintenance_date',
       'service_

In [10]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=5):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

2.4787235260009766


In [236]:
# 56 messy columns are removed
len(messy_columns)

103

In [237]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.6776607036590576


In [238]:
train.shape

(226140, 97)

In [239]:
#Remove the primary key column for data fitting
t1=time.time()
test_ids=test['innovation_challenge_key']
train = train.drop('innovation_challenge_key',axis=1)
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.6395678520202637


In [240]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=5):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.024494647979736328


In [241]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.1072225570678711


In [242]:
'zipcode' in test.columns

False

In [187]:
train=train.drop('zip_code',axis=1)
test=test.drop('zip_code',axis=1)

In [243]:
catcols

['product_transaction_type', 'contract_line_reaction_time_code']

In [244]:
catcols.extend(['sales_hierarchy_level','service_sales_node_base_sales_hierarchy_level'])

In [245]:
scalcols

['sales_hierarchy_level',
 'service_sales_node_base_sales_hierarchy_level',
 'zip_code',
 'contract_line_net_usd_amount',
 'product_net_price',
 'sales_node_renewal_rate',
 'partner_renewal_rate',
 'product_renewal_rate',
 'service_sales_node_installed_base_sales_node_renewal_rate',
 'service_partner_installed_base_partner_renewal_rate']

In [246]:
#scalcols.remove('sales_hierarchy_level')
#scalcols.remove('service_sales_node_base_sales_hierarchy_level')
scalcols.remove('zip_code')

In [247]:
train[scalcols]=train[scalcols].fillna(train[scalcols].mean())
test[scalcols]=test[scalcols].fillna(test[scalcols].mean())

In [248]:
train[catcols]=train[catcols].fillna(train[catcols].mode())
test[catcols]=test[catcols].fillna(test[catcols].mode())

In [249]:
scalcols

['sales_hierarchy_level',
 'service_sales_node_base_sales_hierarchy_level',
 'contract_line_net_usd_amount',
 'product_net_price',
 'sales_node_renewal_rate',
 'partner_renewal_rate',
 'product_renewal_rate',
 'service_sales_node_installed_base_sales_node_renewal_rate',
 'service_partner_installed_base_partner_renewal_rate']

In [250]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

2.9868226051330566


In [251]:
train[scalcols].describe()

,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,226140.000000,2.261400e+05,2.261400e+05
mean,-1.829929e-16,4.524549e-18,1.005455e-17,1.206546e-17,-8.524376e-17,-4.411435e-17,0.000000,5.031204e-17,-8.124158e-18
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002,1.000002e+00,1.000002e+00
min,-2.933243e+00,-2.137117e+01,-1.252316e-01,-2.321716e-01,-5.724046e+00,-3.235729e-01,-0.171298,-2.311610e+00,-6.788500e-02
25%,3.463599e-01,8.693692e-02,-1.252316e-01,-2.321716e-01,0.000000e+00,-6.406592e-03,0.000000,0.000000e+00,-1.663313e-02
50%,3.463599e-01,8.693692e-02,-1.252316e-01,-2.321716e-01,0.000000e+00,-5.496591e-17,0.000000,0.000000e+00,-8.882108e-18
75%,3.463599e-01,8.693692e-02,-1.252316e-01,-1.584198e-01,0.000000e+00,-5.496591e-17,0.000000,0.000000e+00,-8.882108e-18
max,3.463599e-01,8.693692e-02,7.774224e+01,6.648806e+01,7.628503e+01,1.612224e+02,196.387833,9.051475e+01,1.676407e+02


In [252]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

1007.4587190151215


In [253]:
train.columns

Index(['contract_line_reaction_time_code', 'sales_hierarchy_level',
       'service_sales_node_base_sales_hierarchy_level',
       'contract_line_net_usd_amount', 'product_net_price',
       'sales_node_renewal_rate', 'partner_renewal_rate',
       'product_renewal_rate',
       'service_sales_node_installed_base_sales_node_renewal_rate',
       'service_partner_installed_base_partner_renewal_rate',
       ...
       'service_partner_base_minority_owned_business_yorn_N',
       'service_partner_base_minority_owned_business_yorn_Y',
       'service_partner_base_woman_owned_business_yorn_N',
       'service_partner_base_woman_owned_business_yorn_Y',
       'service_partner_base_hubzone_small_business_yorn_N',
       'service_partner_base_hubzone_small_business_yorn_Y',
       'service_partner_base_disabled_veteran_owned_business_yorn_N',
       'service_partner_base_disabled_veteran_owned_business_yorn_Y',
       'distributor_normalized_name_1D5361239B4297AF9198AAEF769C2A4191997D651AF3FF

In [254]:
#print the new columns we got
train.shape

(226140, 10288)

In [255]:
test.shape

(1888195, 10288)

In [204]:
#check data quality now
train.isnull().any().any()

False

In [205]:
#check data quality now
test.isnull().any().any()

False

In [206]:
for col in train.columns:
    print(col)

contract_line_reaction_time_code
sales_hierarchy_level
service_sales_node_base_sales_hierarchy_level
contract_line_net_usd_amount
product_net_price
sales_node_renewal_rate
partner_renewal_rate
product_renewal_rate
service_sales_node_installed_base_sales_node_renewal_rate
service_partner_installed_base_partner_renewal_rate
minor_line_yorn_N
minor_line_yorn_Y
instance_status_Latest-INSTALLED
product_transaction_type_-9999
product_transaction_type_10000
product_transaction_type_10002
installed_product_type_NONSERIALIZED
installed_product_type_SERIALIZED
installed_product_component_yorn_N
installed_product_component_yorn_Y
component_type_CHILD
component_type_PARENT
component_type_PARENT-CHILD
component_type_STANDALONE
scdc_product_yorn_Y
service_classification_ADVANCED SERVICES
service_classification_TECHNICAL SERVICES
contract_line_multi_year_discount_yorn_N
contract_line_multi_year_discount_yorn_Y
contract_line_route_to_market_code_1-TIER
contract_line_route_to_market_code_2-TIER
contrac

In [207]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.24495673179626465


In [208]:
#Dimensions of training data
X_train.shape

(151513, 186)

In [209]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name
clean_col_name=[]
for col in test.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
test.columns = clean_col_name

In [215]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
        'booster' : 'gbtree',
        'max_depth' : int(max_depth),
        'gamma' : gamma,
        'eta' : 0.1,
        'objective' : 'binary:logistic',
        'nthread' : 4,
        'silent' : True,
        'eval_metric': 'logloss',
        'subsample' : max(min(subsample, 1), 0),
        'colsample_bytree' : max(min(colsample_bytree, 1), 0),
        'min_child_weight' : min_child_weight,
        'max_delta_step' : int(max_delta_step),
        'seed' : 1001
        }

    folds = 5
    cv_score = 0

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 20,
                    metrics = 'logloss',
                    show_stdv = True
               )

    val_score = xgbc['test-logloss-mean'].iloc[-1]
    train_score = xgbc['train-logloss-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)
    return (val_score*2) - 1

In [216]:
import xgboost as xgb
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])


In [217]:
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 50.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [218]:
XGB_BO.explore({
              'max_depth':            [3, 8, 3, 8, 8, 3, 8, 3],
              'gamma':                [0.5, 8, 0.2, 9, 0.5, 8, 0.2, 9],
              'min_child_weight':     [0.2, 0.2, 0.2, 0.2, 12, 12, 12, 12],
              'max_delta_step':       [1, 2, 2, 1, 2, 1, 1, 2],
              'subsample':            [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              'colsample_bytree':     [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              })

In [219]:
t1=time.time()
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.495608 val-auc = 0.501026 ( diff = -0.005419 ) train-gini = -0.008785 val-gini = 0.002053
    1 | 04m21s |    0.00205 |             0.6000 |    0.5000 |           1.0000 |      3.0000 |             0.2000 |      0.6000 | 
 Stopped after 500 iterations with train-auc = 0.363188 val-auc = 0.382806 ( diff = -0.019618 ) train-gini = -0.273624 val-gini = -0.234389
    2 | 11m21s |   -0.23439 |             0.8000 |    8.0000 |           2.0000 |      8.0000 |             0.2000 |      0.8000 | 
 Stopped after 500 iterations with train-auc = 0

In [141]:
XGB_BO.res['max']['max_val']

0.15759800000000013

In [142]:
XGB_BO.res['max']['max_params']

{'colsample_bytree': 0.5051029001397185,
 'max_delta_step': 9.995219695530977,
 'max_depth': 2.0605381776650993,
 'min_child_weight': 0.10031720021157042,
 'subsample': 0.5179594185471563}

In [223]:
t1=time.time()
xgb1=xgb.XGBClassifier(max_depth=2,gamma=50.0000,colsample_bytree=0.4,
                       eval_metric='logloss',max_delta_step= 0.0,min_child_weight= 0.0,
                      subsample= 0.4000)
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
t2-t1

24.614728927612305

In [224]:
log_loss(y_valid,pred[:,1])

0.5675290197593071

In [225]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-mincolBO.csv',index=False)

In [ ]:
0.5521,0.5627,0.5533